In [ ]:
# Import necessary libraries
import os, sys, shutil, glob, argparse
import numpy as np
from PIL import Image
from lxml import etree
from scipy.io import loadmat
from datetime import datetime

python_version = sys.version_info.major


###########################################################
##########        WIDER FACE to JSON Conversion       ##########
###########################################################
abs_path = 'datasets'
wider_path = os.path.join(abs_path, 'WIDER/')
json_path = os.path.join(abs_path, 'JSON/')

def make_json_directories():
    """
    Definition: Make directories for json images and annotations.
        Removes previously created json image and annotation directories.
    Returns: None
    """
    if os.path.exists(json_path):
        if python_version == 3:
            prompt = input('Directory already exists. Overwrite? (yes, no): ')
        else:
            prompt = raw_input('Directory already exists. Overwrite? (yes, no): ')
        if prompt == 'no':
            exit(0)
        shutil.rmtree(json_path)
    os.makedirs(json_path)
    os.makedirs(json_path + "images")
    os.makedirs(json_path + "annotations")
    
def copy_images_json():
    """
    Definition: Copy images from images directory of wider dataset
         to image directory in in JSON/images folder.
    """
    for filename in glob.glob(os.path.join(wider_path + "images", "*.*")):
        shutil.copy(filename, json_path + "images/")
   
      
def make_divide_ann():
    """
    Definition: Make directory for wider annotations and populate it with separate ann files.
        Removes previously created wider annotation directory.
    Returns: None
    """
    if os.path.exists(wider_path+'divide_ann'):
        if python_version == 3:
            prompt = input('Directory already exists. Overwrite? (yes, no): ')
        else:
            prompt = raw_input('Directory already exists. Overwrite? (yes, no): ')
        if prompt == 'no':
            exit(0)
        shutil.rmtree(wider_path+'divide_ann')
    os.makedirs(wider_path + "divide_ann")
    
    filename = os.path.join(wider_path + 'annotations/ann.txt') 
   
    file = open(filename)
    # Iterate through wider annotation data
    for line in file:
        if line[-5:-1] == ".jpg":
            f= open(wider_path+'divide_ann/' + line.split("/")[1].split(".jpg")[0]+'.txt',"w+")
        if line[-5:-1] != ".jpg":
            l = line.split(" ")
            if len(l) < 2:
                file_lines_count =int(line.split("/")[0])
            if len(l) > 2:
                f.write(line)
    file.close()
def parse():
    """
    Definition: Parses divide_ann file to extract bounding boxcoordintates.
    """
    objects = []   
    object_info = {}
    for f in os.listdir(wider_path + "divide_ann/"):
        lfile = open(wider_path + "divide_ann/"+f)
        object_info['filename'] = f.split(".")[0]+".jpg"
        object_info['objects'] = []
        coords = []
        coor = []
        # Load object bounding boxes.
        for line in lfile:
            coor = line.split(" ")[0:4]
            x1 = int(coor[0])
            y1 = int(coor[1])
            x2 = int(coor[0])+int(coor[2])
            y2 = int(coor[1])+int(coor[3])
            
            tmp = [x1,y1,x2,y2]
            coords.append(tmp)
            person_info = {'class_name':'Person'}
            person_info ['bounding_box'] = tmp
            object_info['objects'].append(person_info)
            objects.append(object_info)
            tmp = []
            for j in objects:
                f= open(json_path+'annotations/'+j["filename"].split(".jpg")[0]+'.json',"w+")
                f.write(str(j))
                f.close()
def json():
    make_json_directories()
    copy_images_json()
    make_divide_ann()
    parse()
    
json()


